## Run and Evaluate DeepSeek-R1 (distilled model) with Ollama and OpenAI's simple-evals
### Notebook Walkthrough
Author: Kenneth Leung
___

### (1) Installation Instructions
- Download and install Ollama from https://ollama.com/download
- To start Ollama, either open the Ollama app on your local machine, or run `ollama serve` in the terminal.
- We will be working with the distilled DeepSeek-R1-Distill-Qwen-7B model. Pull the model by running the following command in terminal:
  - `ollama pull deepseek-r1:7b`
- Once done, we return to this notebook to continue with the Python codes

___
### (2) Initial Setup

In [3]:
import ollama
import time
from simple_evals.gpqa_eval import GPQAEval

from utils.utils import load_config
from utils.samplers.ollama_sampler import OllamaSampler

In [2]:
config = load_config("src/config/config.yaml")
config

{'MODEL_NAME': 'deepseek-r1:7b'}

In [4]:
# Confirm that our model has been downloaded
print(ollama.list())

models=[Model(model='deepseek-r1:1.5b', modified_at=datetime.datetime(2025, 3, 6, 11, 33, 42, 130232, tzinfo=TzInfo(+08:00)), digest='a42b25d8c10a841bd24724309898ae851466696a7d7f3a0a408b895538ccbc96', size=1117322599, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='1.8B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:7b', modified_at=datetime.datetime(2025, 3, 6, 11, 20, 52, 985187, tzinfo=TzInfo(+08:00)), digest='0a8c266910232fd3291e71e5ba1e058cc5af9d411192cf88b6d30e92b6e73163', size=4683075271, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='7.6B', quantization_level='Q4_K_M'))]


___
### (3) Single test run

In [4]:
# prompt = "What is the capital of China?"

prompt = """
You are an advanced AI assistant analyzing an alien civilization’s mathematical system. 
They use an unfamiliar number system, and their number patterns follow unknown rules. 
You receive the following number sequences and must determine the missing number:

Sequences:
3, 6, 11, 18, 27, ?
2, 6, 12, 20, 30, ?
5, 10, 18, 30, 47, ?
Rules:
The aliens do not use base-10 but instead follow their own logical sequence.
Each sequence follows a hidden pattern based on an unknown mathematical principle.
You must determine the next number in each sequence and briefly explain the reasoning behind it.

Ensure you initiate your response with "<think>\n at the beginning of your output.
"""

In [ ]:
start_time = time.time()
response: ollama.ChatResponse = ollama.chat(model=config["MODEL_NAME"], 
                                            messages=[
                                              {'role': 'user',
                                               'content': prompt},
                                            ])
end_time = time.time()
execution_time = end_time - start_time
minutes = int(execution_time // 60)
seconds = execution_time % 60
if minutes > 0:
    print(f"\nExecution Time: {minutes} min {seconds:.2f} sec\n")
else:
    print(f"\nExecution Time: {seconds:.2f} sec\n")

print(response['message']['content'])


Execution Time: 2 min 8.55 sec

<think>
Okay, so I need to figure out what the capital of China is. Hmm, I remember learning about capitals in school when we studied countries and their capitals. Let me think... I'm pretty sure it's a city that's both the political center and where the president lives.

I know that other countries like the United States have Washington D.C. as their capital, which isn't on the east coast but kind of mid-northwest in the U.S.A. Then there's Canada with Ottawa, which is right across the border from where I live in the southern part of Ontario. So maybe China has a similar situation?

Wait, I'm pretty sure that besides Beijing, there's also Shanghai and Shenzhen. I think Shanghai is the economic capital because it's big and has all those financial activities, while Shenzhen is more modern and tech-oriented. But which one is considered the political or official capital? I believe it's Beijing where all the government stuff happens, like the President live

___

### (4) Initiate GPQA Evaluation

In [ ]:
start_time = time.time()

# Load the Ollama wrapper that wraps ollama.chat() to format prompts correctly and retrieve responses for GPQA eval
ollama_sampler = OllamaSampler(model_name=config["MODEL_NAME"])

# Instantiate the GPQAEval class for evaluation
gpqa_eval = GPQAEval(n_repeats=config["EVAL_N_REPEATS"],
                     num_examples=config["EVAL_N_EXAMPLES"], 
                     variant=config["GPQA_VARIANT"])

# Run GPQA evaluation
results = gpqa_eval(ollama_sampler)

end_time = time.time()
elapsed_seconds = end_time - start_time
minutes, seconds = divmod(
    elapsed_seconds, 60
)

# The returned results is an EvalResult which includes a list of SingleEvalResult
# and aggregated metrics. Print metrics:
print("Overall Evaluation Metrics:")
print(results.metrics)
print(f"Total Execution Time: {int(minutes)} min {seconds:.2f} sec")

In [ ]:
results